In [21]:
from tensor import Tensor
import numpy as np 
from functools import partialmethod

In [5]:
x_init_ = np.random.randn(1, 3).astype(np.float32)
print(x_init_)

[[-0.8195286  -0.47795168  0.08329567]]


In [6]:
x_ = Tensor(x_init_)

dir(x_)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_ctx',
 'data',
 'grad',
 'mul']

# Building Tensor 

In [100]:
class OSTensor:
    def __init__(self, data):
        self.data = data
    
    def __repr__(self) -> str:
        return f"OSTensor with data: {self.data}"

## Add the multiplication to the OSTensor

In [101]:
class osFunction:
    def apply(self, osClass, data):
        val = osClass.forward(self.data, data)
        t_ = OSTensor(val)
        return t_

# Define the multiplication class 
class osMul:
    @staticmethod
    def forward(x, y):
        return x * y
    
class osAdd:
    @staticmethod
    def forward(x, y):
        return x + y.data

setattr(OSTensor, 'Multiply', partialmethod(osFunction.apply, osMul))
setattr(OSTensor, 'Add', partialmethod(osFunction.apply, osAdd))

t1 = OSTensor(3)
t2 = OSTensor(5)

t3 = t1.Multiply(5)
print(t3)

t4 = t1.Add(t2)
print(t4)

OSTensor with data: 15
OSTensor with data: 8


### Need to register the Multiplication in OSTensor class 

In [6]:

setattr(OSTensor, 'Mul', osMul.forward)


In [7]:
x_init = np.random.randn(1, 3).astype(np.float32)

x = OSTensor(x_init)

x.Mul(2)

in Mul forward <__main__.OSTensor object at 0x107d029d0> 2


TypeError: unsupported operand type(s) for *: 'OSTensor' and 'int'

# Scrap

In [8]:
class A:
    def __init__(self, arg, *x):
        self.arg = arg
        self.tensors = x
        print(f" in the A ")

    def __repr__(self):
        return f"A with {self.arg}, {self.tensors}"

class B:
    def apply(self, arg, *x):
        self.ctx = A(arg, self, 'arg3', 2, 3, 4, 4, *x)


b = B()
b.apply('arg1', x_init_)
print(b.ctx)


print(b.ctx.tensors[0])










 in the A 
A with arg1, (<__main__.B object at 0x112a02eb0>, 'arg3', 2, 3, 4, 4, array([[-0.8195286 , -0.47795168,  0.08329567]], dtype=float32))


In [59]:
setattr(A, 'newmethod', lambda x: 2*x)

class T:
    def __init__(self, data):
        self.data = data
    def __repr__(self):
        return f"Data in T: {self.data}"
       

class testFunction:
    def apply(self, Mul, data):
        print(f"I am in the apply method in testFunction class: {data}")
        print(self.data)
        val = Mul.forward(self.data, data)
        ret = T(val)
        return ret


class testMul(testFunction):
    @staticmethod
    def forward(x, y):
        print(f"x: {x}, y: {y}")
        return x * y



setattr(T, 'Multiply', partialmethod(testFunction.apply, testMul))



t1 = T(5)

t2 = t1.Multiply(30)


print(t1)
print(t2)


dir(t1)

I am in the apply method in testFunction class: 30
5
x: 5, y: 30
Data in T: 5
Data in T: 150


['Multiply',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'data']

In [30]:
partialmethod?

Init signature: partialmethod(func, /, *args, **keywords)
Docstring:     
Method descriptor with partial application of the given arguments
and keywords.

Supports wrapping existing descriptors and handles non-descriptor
callables as instance methods.
File:           ~/anaconda3/envs/tinygrad/lib/python3.8/functools.py
Type:           type
Subclasses:     

In [17]:
A.newmethod(2)

4